# 0. Importing Necessary Packages

In [116]:
# Printing the information of Python, IPython, OS, and the generation date.
%load_ext version_information
%version_information

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 2.7.17 64bit [GCC 7.3.0]
IPython 5.8.0
OS Linux 5.8.18 100.fc31.x86_64 x86_64 with fedora 31 Thirty_One
Thu Apr 07 16:45:25 2022 KST

In [117]:
# Printing the versions of packages
from importlib_metadata import version
for pkg in ['numpy', 'matplotlib', 'pandas', 'astropy', 'pyraf']:
    print(pkg+": ver "+version(pkg))

numpy: ver 1.16.5
matplotlib: ver 2.2.3
pandas: ver 0.24.2
astropy: ver 2.0.16
pyraf: ver 2.1.15


In [118]:
# matplotlib backend
%matplotlib notebook

# importing necessary modules
import time
import numpy as np
import glob, os
from matplotlib import pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.stats import sigma_clipped_stats

current_dir = os.getcwd()
dir_iraf = os.path.abspath('../')    # where 'login.cl' file is located

os.chdir(dir_iraf)
from pyraf import iraf
os.chdir(current_dir)

# 1. Selecting PSF Stars in the Images

In [119]:
dir_img = "HolmIX/"
imglist = sorted(glob.glob(dir_img+"*.fits"))
n_img = len(imglist)
imglist

['HolmIX/HolmIX_g.fits',
 'HolmIX/HolmIX_g.psf.1.fits',
 'HolmIX/HolmIX_g.psf.2.fits',
 'HolmIX/HolmIX_g.spsf.1.fits',
 'HolmIX/HolmIX_i.fits']

In [121]:
iraf.imexamine()

Killing IRAF task `imexamine'


IrafError: Error running IRAF task imexamine
IRAF task terminated abnormally
ERROR (742, "Cannot open device (node!imtool,,8192,8192)")


In [5]:
# You can also run this command in terminal.
ds9_options = "-scalemode zscale -scale lock yes -frame lock image "
names = ""
for i in np.arange(n_img):
    names += imglist[i]+" "
ds9_command = "ds9 "+ds9_options+names+"&"
print('Running "'+ds9_command+'" in the terminal...')
os.system(ds9_command)

Running "ds9 -scalemode zscale -scale lock yes -frame lock image HolmIX/HolmIX_g.fits HolmIX/HolmIX_i.fits &" in the terminal...


0

In [64]:
p0_x, p0_y = np.loadtxt(dir_img+"init_point.reg").T
dat_phot = np.genfromtxt("HolmIX/tmp", dtype='float', encoding="ascii",
                         names=("ID","XCENTER","YCENTER","MAG","MSKY"))

In [39]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import CartesianRepresentation

In [78]:
coord_regr = SkyCoord(p0_x*u.m, p0_y*u.m, 0.*u.m, representation='cartesian')
coord_phot = SkyCoord(dat_phot['XCENTER']*u.m, dat_phot['YCENTER']*u.m, 0.*u.m, representation='cartesian')

In [79]:
coord_phot

<SkyCoord (ICRS): (x, y, z) in m
    [( 958.971,    7.11 , 0.), ( 173.533,   11.188, 0.),
     ( 573.037,   11.084, 0.), ..., (1002.524, 2295.431, 0.),
     (1444.148, 2295.882, 0.), (1445.735, 2296.83 , 0.)]>

In [80]:
from astropy.coordinates import matching

In [88]:
idx, sep2d, dist3d = matching.match_coordinates_sky(coord_regr, coord_phot)

In [90]:
coord_regr

<SkyCoord (ICRS): (x, y, z) in m
    [( 135.45, 2148.3 , 0.), (  43.97, 1644.04, 0.),
     ( 255.01, 1691.56, 0.), ( 505.03, 1740.82, 0.),
     ( 481.35, 1424.61, 0.), ( 394.1 , 1320.08, 0.),
     ( 114.17,  565.47, 0.), ( 813.4 ,   57.42, 0.),
     (1215.78,  271.26, 0.), (1229.25,  526.65, 0.),
     (1941.28,  407.71, 0.), (2232.63,  648.03, 0.),
     (2083.03, 1387.91, 0.), (2400.55, 2285.54, 0.),
     (1300.11, 2240.43, 0.)]>

In [91]:
coord_phot[idx]

<SkyCoord (ICRS): (x, y, z) in m
    [( 135.457, 2148.271, 0.), (  43.975, 1644.055, 0.),
     ( 255.043, 1691.576, 0.), ( 505.043, 1740.849, 0.),
     ( 481.344, 1424.64 , 0.), ( 394.096, 1320.088, 0.),
     ( 114.151,  565.501, 0.), ( 813.376,   57.402, 0.),
     (1215.821,  271.255, 0.), (1229.25 ,  526.681, 0.),
     (1941.284,  407.765, 0.), (2232.694,  648.085, 0.),
     (1394.986,  929.466, 0.), (1302.321, 1239.92 , 0.),
     (1300.096, 2240.482, 0.)]>

In [93]:
idx

array([5267, 4923, 4954, 4983, 4538, 4285,  458,   23,  177,  383,  256,
        668, 2130, 4027, 5324])

In [105]:
f = open(dir_img+"HolmIX_g.pst.0", "w")
f.write("#N ID    XCENTER   YCENTER   MAG         MSKY \n")
f.write("#U ##    pixels    pixels    magnitudes  counts \n")
f.write("#F %-9d  %-10.3f   %-10.3f   %-12.3f     %-15.7g\n")
f.write("#\n")
for i in idx:
    f.write("{0:<9d} {1:<10.3f} {2:<10.3f} {3:<12.3f} {4:<15.7g}\n".format(int(dat_phot['ID'][i]),
                                                                 dat_phot['XCENTER'][i],
                                                                 dat_phot['YCENTER'][i],
                                                                 dat_phot['MAG'][i],
                                                                 dat_phot['MSKY'][i]))
f.close()#F %-9d  %-10.3f   %-10.3f   %-12.3f     %-15.7g 

python 3 representation_type|

In [6]:
iraf.noao()
iraf.digiphot()
iraf.daophot()

In [57]:
iraf.txdump("HolmIX/HolmIX_g.mag", "ID,XCENTER,YCENTER,MAG,MSKY","yes", Stdout="HolmIX/tmp")

In [62]:
np.genfromtxt("HolmIX/tmp", dtype='float', encoding="ascii")

array([[ 1.000000e+00,  9.589710e+02,  7.110000e+00,  2.901700e+01,
        -1.621209e-03],
       [ 2.000000e+00,  1.735330e+02,  1.118800e+01,  2.479700e+01,
        -1.679210e-03],
       [ 3.000000e+00,  5.730370e+02,  1.108400e+01,  2.574800e+01,
         2.018862e-03],
       ...,
       [ 5.343000e+03,  1.002524e+03,  2.295431e+03,  2.375300e+01,
         3.005540e-03],
       [ 5.344000e+03,  1.444148e+03,  2.295882e+03,  2.364100e+01,
        -3.548646e-03],
       [ 5.345000e+03,  1.445735e+03,  2.296830e+03,           nan,
        -3.360884e-03]])

In [12]:
img_g, hdr_g = fits.getdata(imglist[0], header=True)
img_i, hdr_i = fits.getdata(imglist[1], header=True)

In [19]:
med_fwhm_g, med_fwhm_i = np.loadtxt(dir_img+"fwhms.txt").T
print(med_fwhm_g, med_fwhm_i)
sky_value, sky_sigma = np.loadtxt(dir_img+"sky.txt")
print(sky_value, sky_sigma)

(3.845, 3.85)
(array([0.006295, 0.01383 ]), array([0.03368, 0.07592]))


In [20]:
# PSF task (interative)
band = ["g", "i"]
fwhms = [med_fwhm_g, med_fwhm_i]

start_time = time.time()
for i, img in enumerate([img_g, img_i]):
    imgname = imglist[i].split(".fits")[0]
    magfile = imgname+".mag"
    psffile = imgname+".psf.1.fits"
    pstfile = imgname+".pst.1"
    psgfile = imgname+".psg.1"
    os.system("rm -rfv "+psffile+" "+pstfile+" "+psgfile)
    
    kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
              "datamax":100.0, "readnoi":5.0, "epadu":3.0, "itime":1.0,
              "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i],
              "function":"auto", "varorder":0, "interactive":False}

    iraf.psf(image=imglist[i], photfile=magfile, 
             psfimage=psffile, opstfile=pstfile, groupfile=psgfile, **kwargs)
    
end_time = time.time()
print("\n--- PSF tasks take {0:.4f} sec ---".format(end_time-start_time))


Analytic psf function(s) (auto): 	Analytic psf function(s): auto
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (15.38): 	New psf radius: 15.38 scale units 15.38 pixels
Fitting radius in scale units (3.845): 	New fitting radius: 3.845 scale units 3.845 pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (100.) (CR or value): 	New maximum good data value: 100. counts




Computing PSF for image: HolmIX/HolmIX_g.fits
5345 stars read from HolmIX/HolmIX_g.mag

Star at 478.283,1426.313  not found in the photometry file
Star 4539 has been added to the PSF star list
	X:  481.34 Y: 1424.64  Mag:  20.664  Dmin: -0.04916718  Dmax: 19.80373
Star 4539 is already a PSF star
Star 4539 is already a PSF star
Star 4286 has been added to the PSF star list
	X:  394.10 Y: 1320.09  Mag:  21.319  Dmin: -0.04231523  Dmax: 11.30819
Star 384 has been added to the PSF star list
	X: 1229.25 Y:  526.68  Mag:  20.797  Dmin: -0.05696667  Dmax: 17.86062
Star 178 has been added to the PSF star list
	X: 1215.82 Y:  271.26  Mag:  22.297  Dmin: -0.05186392  Dmax: 4.669229
Star 121 rejected by user
Star 24 has been added to the PSF star list
	X:  813.38 Y:   57.40  Mag:  20.947  Dmin: -0.04667079  Dmax: 15.37144
Trying function gauss norm scatter = 0.02523643
Trying function lorentz norm scatter = 0.03831252
Trying function moffat15 norm scatter = 0.02225554
Trying function moffat25 no

error: (25, 'Inappropriate ioctl for device')

In [115]:
i = 0
kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
          "datamax":100.0, "readnoi":5.0, "epadu":3.0, "itime":1.0,
          "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i],
          "function":"auto", "varorder":0, "interactive":False}
os.system("rm -rfv "+psffile+" "+pstfile+" "+psgfile)
iraf.psf(image=imglist[i], photfile=magfile, pstfile="HolmIX/HolmIX_g.pst.0",
         psfimage=psffile, opstfile=pstfile, groupfile=psgfile, **kwargs)


Analytic psf function(s) (auto): 	Analytic psf function(s): auto
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (15.38): 	New psf radius: 15.38 scale units 15.38 pixels
Fitting radius in scale units (3.845): 	New fitting radius: 3.845 scale units 3.845 pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (100.) (CR or value): 	New maximum good data value: 100. counts


Computing PSF for image: HolmIX/HolmIX_g.fits
5345 stars read from HolmIX/HolmIX_g.mag

Star 5268 has been added to the PSF star list
	X:  135.46 Y: 2148.27  Mag:  21.601  Dmin: -0.06291332  Dmax: 8.161823
Star 4924 has been added to the PSF star list
	X:   43.97 Y: 1644.06  Mag:  21.062  Dmin: -0.04324778  Dmax: 14.36388
Star 4955 has been added to the PSF star list
	X:  255.04 Y: 1691.58  Mag:  21.390  Dmin: -0.0726084  Dmax: 10.52991
Star 4984 has been added to the PSF star list
	X:  505.04 Y: 1740.85  Mag:  21.994  Dm

In [114]:
kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
          "datamax":100.0, "readnoi":5.0, "epadu":3.0, "itime":1.0,
          "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i]}
alsfile = imgname+".als.1"
rejfile = imgname+".arj.1"
subimage = imgname+".sub.1.fits"

iraf.allstar(image=imglist[i], photfile=psgfile, psfimage=psffile,
             allstarfile=alsfile, rejfile=rejfile, subimage=subimage, **kwargs)


Recenter the stars (yes): 	Recenter the stars: yes
Use group sky values (yes): 	Use group sky values: yes
Refit the sky (no): 	Refit the sky: no
Psf radius in scale units (15.38): 	New psf radius: 15.38 scale units 15.38 pixels
Fitting radius in scale units (3.845): 	New fitting radius: 3.845 scale units 3.845 pixels
Maximum group size in number of stars (60): 	New maximum group size: 60 stars
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (100.) (CR or value): 	New maximum good data value: 100. counts



Killing IRAF task `allstar'


IrafError: Error running IRAF task allstar
IRAF task terminated abnormally
ERROR (1, "Table `HolmIX/HolmIX_g.psg.2' does not exist or cannot be opened.")


In [113]:
os.system("rm -rfv "+imgname+".sub.1.fits")
iraf.substar(image=imglist[i], photfile=alsfile, exfile=pstfile, psfimage=psffile,
             subimage=imgname+".sub.1.fits", **kwargs)

KeyError: "Error in parameter 'interactive' for task substar\n'Key interactive not found'"

In [111]:
iraf.seepsf(psffile, "HolmIX/HolmIX_g.spsf.1.fits")

In [112]:
magfile = imgname+".mag"
psffile = imgname+".psf.2.fits"
pstfile = imgname+".pst.2"
psgfile = imgname+".psg.2"
os.system("rm -rfv "+psffile+" "+pstfile+" "+psgfile)

i = 0
kwargs = {"fwhmpsf":fwhms[i], "sigma":sky_sigma[i],
          "datamax":100.0, "readnoi":5.0, "epadu":3.0, "itime":1.0,
          "psfrad":4.0*fwhms[i], "fitrad":1.0*fwhms[i],
          "function":"auto", "varorder":0, "interactive":False}
os.system("rm -rfv "+psffile+" "+pstfile+" "+psgfile)
iraf.psf(image=imgname+".sub.1.fits", photfile=magfile, pstfile="HolmIX/HolmIX_g.pst.1",
         psfimage=psffile, opstfile=pstfile, groupfile=psgfile, **kwargs)


Analytic psf function(s) (auto): 	Analytic psf function(s): auto
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (15.38): 	New psf radius: 15.38 scale units 15.38 pixels
Fitting radius in scale units (3.845): 	New fitting radius: 3.845 scale units 3.845 pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (100.) (CR or value): 	New maximum good data value: 100. counts



Killing IRAF task `psf'


IrafError: Error running IRAF task psf
IRAF task terminated abnormally
ERROR (1113, "FXF: must specify which FITS extension (HolmIX/HolmIX_g.sub.1.fits)")
